<span style="box-sizing: inherit; font-weight: 700; color: rgb(34, 34, 34); font-family: Arial, Helvetica, sans-serif; font-size: 17px; background-color: rgb(255, 255, 255);">Average Salary of all employees</span>

In [1]:
select avg(salary) as avg_salary
from Employees;

(1 row affected)

Total execution time: 00:00:00.033

avg_salary
6400


<span style="box-sizing: inherit; font-weight: 700; color: rgb(34, 34, 34); font-family: Arial, Helvetica, sans-serif; font-size: 17px; background-color: rgb(255, 255, 255);">Average Salary of all employees Department wise</span>

In [2]:
select department_id, avg(salary) as avg_salary
from Employees
group by department_id

(5 rows affected)

Total execution time: 00:00:00.010

department_id,avg_salary
1,5500
2,6500
3,5000
4,8000
5,7000


<span style="box-sizing: inherit; font-weight: 700; color: rgb(34, 34, 34); font-family: Arial, Helvetica, sans-serif; font-size: 17px; background-color: rgb(255, 255, 255);">Displaying average salary against each employee record</span>

In [5]:
select employee_id, e.department_id, avg_salary
from Employees e 
left join (select department_id, avg(salary) as avg_salary
    from Employees
    group by department_id) ae 
    on e.department_id = ae.department_id

(10 rows affected)

Total execution time: 00:00:00.011

employee_id,department_id,avg_salary
1,1,5500
2,2,6500
3,1,5500
4,3,5000
5,4,8000
6,5,7000
7,2,6500
8,3,5000
9,5,7000
10,4,8000


In [7]:
select employee_id, department_id,
avg(salary) over (partition by department_id) as avg_salary
from Employees
order by employee_id

(10 rows affected)

Total execution time: 00:00:00.010

employee_id,department_id,avg_salary
1,1,5500
2,2,6500
3,1,5500
4,3,5000
5,4,8000
6,5,7000
7,2,6500
8,3,5000
9,5,7000
10,4,8000


## Calculate total sum of salary department wise

In [9]:
select employee_id,
department_id,
sum(salary) over (PARTITION by department_id) as sum_salary
from Employees

(10 rows affected)

Total execution time: 00:00:00.014

employee_id,department_id,sum_salary
3,1,11000
1,1,11000
2,2,13000
7,2,13000
8,3,10000
4,3,10000
5,4,16000
10,4,16000
6,5,14000
9,5,14000


## Calculate cumulative sum of salary department wise

In [11]:
SELECT employee_id,
department_id,
salary,
sum(salary) over (PARTITION by department_id order by employee_id) as cum_sum_salary
from Employees

(10 rows affected)

Total execution time: 00:00:00.015

employee_id,department_id,salary,cum_sum_salary
1,1,5000,5000
3,1,6000,11000
2,2,10000,10000
7,2,3000,13000
4,3,5000,5000
8,3,5000,10000
5,4,8000,8000
10,4,8000,16000
6,5,7000,7000
9,5,7000,14000


## Calculate cumulative sum of the organization

In [12]:
select employee_id,
department_id,
salary,
sum(salary) over (order by employee_id) as cum_sum_salary
from Employees

(10 rows affected)

Total execution time: 00:00:00.010

employee_id,department_id,salary,cum_sum_salary
1,1,5000,5000
2,2,10000,15000
3,1,6000,21000
4,3,5000,26000
5,4,8000,34000
6,5,7000,41000
7,2,3000,44000
8,3,5000,49000
9,5,7000,56000
10,4,8000,64000


## Calculate Cumulative average of the salary department wise

In [15]:
select employee_id,
department_id,
salary,
avg(salary) over (PARTITION by department_id order by employee_id) as cum_avg_salary
from Employees

(10 rows affected)

Total execution time: 00:00:00.009

employee_id,department_id,salary,cum_avg_salary
1,1,5000,5000
2,1,10000,7500
3,1,6000,7000
7,2,3000,3000
4,3,5000,5000
8,3,5000,5000
5,4,8000,8000
10,4,8000,8000
6,5,7000,7000
9,5,7000,7000


## Calculate average of salary for current and previous record department wise

In [18]:
select employee_id,
department_id,
salary,
avg(salary) over (PARTITION by department_id order by employee_id rows 1 preceding) as avg_salary
from Employees

(10 rows affected)

Total execution time: 00:00:00.009

employee_id,department_id,salary,avg_salary
1,1,5000,5000
2,1,10000,7500
3,1,6000,8000
7,2,3000,3000
4,3,5000,5000
8,3,4500,4750
5,4,8000,8000
10,4,3000,5500
6,5,4000,4000
9,5,7000,5500


## Find the oldest joinee department wise using LAG Analytic function

In [19]:
alter table Employees add hire_date DATE;

Commands completed successfully.

Total execution time: 00:00:00.172

In [24]:
select * 
from
(select employee_id,
department_id,
hire_date,
lag(hire_date) over (partition by department_id order by hire_date) as last_hire_date
from Employees) e
where last_hire_date is null;

(5 rows affected)

Total execution time: 00:00:00.011

employee_id,department_id,hire_date,last_hire_date
3,1,2018-03-03,NULL
7,2,2023-10-03,NULL
8,3,2018-05-13,NULL
10,4,2016-10-03,NULL
9,5,2020-09-19,NULL


## Find the newest joinee department wise using LAG Analytic function

In [26]:
select * 
from (select employee_id,
department_id,
hire_date,
lag(hire_date) over (partition by department_id order by hire_date desc) as new_hire_date
from Employees) eh
where new_hire_date is null;

(5 rows affected)

Total execution time: 00:00:00.011

employee_id,department_id,hire_date,new_hire_date
2,1,2019-09-05,NULL
7,2,2023-10-03,NULL
4,3,2020-01-23,NULL
5,4,2023-01-16,NULL
6,5,2021-05-06,NULL


## Find the oldest joinee department wise using LEAD Analytic function

In [29]:
select * from (select employee_id,
department_id,
hire_date,
lead(hire_date) over (PARTITION by department_id order by hire_date desc) as new_hire_date
from Employees) el 
where new_hire_date is null

(5 rows affected)

Total execution time: 00:00:00.010

employee_id,department_id,hire_date,new_hire_date
3,1,2018-03-03,NULL
7,2,2023-10-03,NULL
8,3,2018-05-13,NULL
10,4,2016-10-03,NULL
9,5,2020-09-19,NULL


## Find the newest joinee department wise using LEAD Analytic function

In [31]:
select * 
from (select employee_id,
department_id,
hire_date,
lead(hire_date) over (partition by department_id order by hire_date) as new_hire_date
from Employees) el 
where new_hire_date is null;

(5 rows affected)

Total execution time: 00:00:00.011

employee_id,department_id,hire_date,new_hire_date
2,1,2019-09-05,NULL
7,2,2023-10-03,NULL
4,3,2020-01-23,NULL
5,4,2023-01-16,NULL
6,5,2021-05-06,NULL


### Find employee with MAX salary department wise

In [37]:
select *
from (select employee_id,
department_id,
salary,
rank() over (partition by department_id order by salary desc) as max_salary
from Employees) em
where max_salary = 1

(5 rows affected)

Total execution time: 00:00:00.011

employee_id,department_id,salary,max_salary
2,1,10000,1
7,2,3000,1
4,3,5000,1
5,4,8000,1
9,5,7000,1


### Find employee with MIN salary department wise

In [39]:
select * 
from (select employee_id,
department_id,
salary,
rank() over (partition by department_id order by salary) as min_salary
from Employees) em
where min_salary = 1

(5 rows affected)

Total execution time: 00:00:00.011

employee_id,department_id,salary,min_salary
1,1,5000,1
7,2,3000,1
8,3,4500,1
10,4,3000,1
6,5,4000,1


## Find the difference between the salary of an employee and max salary of the employee in the department

In [46]:
select employee_id, department_id, salary,
max(salary) over (partition by department_id) as max_salary,
(max(salary) over (partition by department_id)-salary) as diff_salary
from Employees
order by employee_id

(10 rows affected)

Total execution time: 00:00:00.011

employee_id,department_id,salary,max_salary,diff_salary
1,1,5000,10000,5000
2,1,10000,10000,0
3,1,6000,10000,4000
4,3,5000,5000,0
5,4,8000,8000,0
6,5,4000,7000,3000
7,2,3000,3000,0
8,3,4500,5000,500
9,5,7000,7000,0
10,4,3000,8000,5000


## Find employee with MAX salary department wise without using RANK or DENSE\_RANK

In [49]:
select * 
from (select employee_id, department_id, salary,
max(salary) over (partition by department_id) as max_salary,
(max(salary) over (partition by department_id)-salary) as diff_salary
from Employees) em
where diff_salary = 0

(5 rows affected)

Total execution time: 00:00:00.011

employee_id,department_id,salary,max_salary,diff_salary
2,1,10000,10000,0
7,2,3000,3000,0
4,3,5000,5000,0
5,4,8000,8000,0
9,5,7000,7000,0
